## Homework 09:  Parallel Programming 02

## Due Date: Apr 27, 2020, 08:00am

#### Firstname Lastname: 

#### E-mail: 

#### Enter your solutions and submit this notebook

---

**Problem 1 (40p)**

In this problem the goal is to calculate the mean and standard deviation of a large list of numbers by using Reduction as one of Collective Operations, see Lecture 11. 


Consider $N = 256000$ random variables uniform on $[0, 1]$, call these $x_0, x_1, \dots, x_{N - 1}$.  


Write an MPI program with $N=16$ processes that outputs the average and standard deviation of $x_0, x_1, \dots, x_{N - 1}$.


To simplify the problem, let one process say `Process 0`, initialize $N$ random uniform variables. 


**Instructions:** 
Your program should use MPI4PY and collective operations. 
Save your program as 2020_spring_sol09_pr01.py and run it from the terminal as:

```
mpirun -n 16 python3 2020_spring_sol09_pr01.py
```

here `python3` is your path to Python.


---
**Problem 2 (60p)**

In this problem the goal is to demonstrate how one can use a Domain Decomposition and  Collective Operations. 

Consider the exponential distribution $X \sim \textrm{Exp}(1)$ with the unit mean. Find numerical approximations of moments of the exponential random variable. 

That is, for a random variable $X$ with the distribution $f(x) = e^{-x}$ for $x \geq 0$, compute the first $15$ moments, where the $k$-th moment is defined as:
$$I_k = \int_{0}^{\infty} x^k f(x) dx.$$


Your program should use MPI parallel collective instructions, where the integration is performed in parallel over $N=16$ processes, over a finite range $[0, M)$, where $M=1000$, with $N = 16$ partitions and $1000$ increments per partition, see Lecture 10 and 11.

Provide evaluations of $J_1, J_2, \dots, J_{15}$, where $$J_k = \int_{0}^{M} x^k f(x) dx.$$


**Instructions:** 

Save your program as 2020_sol09_pr02.py; and run it from the terminal as:

```
mpirun -n 16 python3 2020_spring_sol09_pr01.py
```

here `python3` is your path to Python.


**Bonus Question (10 points):** 

What is the value of $I_k$, as a function of $k$? How can it be derived?

### Solution Problem 1 

### This problem was demonstrated in Lab.11


In [1]:
%%writefile 2020_spring_sol09_pr01.py


from mpi4py import MPI
import numpy as np


comm = MPI.COMM_WORLD

my_rank = comm.Get_rank()
p = comm.Get_size()

n = 256000

if my_rank == 0:
    data = np.random.random(n).astype('f')       
    data = data.reshape(p, int(n/p))    
    
else:
    data = None

recvbuf = np.empty(int(n/p), dtype='f')

#Scatter the numbers to all processes
comm.Scatter(data,recvbuf, root = 0)

local_sum = sum(recvbuf)
print('Process {:02d}, local sum = {:.4f}'.format(my_rank, local_sum))

#Reduce local sums into a global sum to calculate the mean
global_sum = comm.allreduce(local_sum, MPI.SUM)
mean = global_sum / n

#Compute the local sum of squared differences from the mean

local_sq_diff = sum((num - mean) ** 2 for num in recvbuf)

#Reduce the global sum of the squared differences to the root process
global_sq_diff = comm.reduce(local_sq_diff, MPI.SUM, 0)

if my_rank == 0:
    stddev = np.sqrt(global_sq_diff / n)
    print('\nMean: {:.4f}, Standard deviation: {:.4f}'.format(mean, stddev))
    print('Standard deviation using numpy: {:.4f}'.format(np.std(data)))


Overwriting 2020_spring_sol09_pr01.py


### Solution Problem 2

In [2]:
%%writefile 2020_spring_sol09_pr02.py

import numpy as np
from numpy import pi, exp, sqrt
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

def integral(ar, h, n, K):
    
    integ = np.zeros(K)
    for j in range(n):
        arj = ar + (j + 0.5) * h
        integ += np.array([arj ** k * exp(- arj) for k in range(1, K + 1)])
    
    # multiply with h so to get 
    # the approximate area under the curve
    integ *= h
    return integ


K = 15
M = 1000
n = 1000

my_int = np.zeros(K)
integral_sum = np.zeros(K)
n = np.array(10_000)

# Compute partition
h = M / (n * size) 
ar = rank * h * n

my_int = integral(ar, h, n, K)
# Send partition back to root process, computing sum across all partitions
comm.Reduce(my_int, integral_sum, MPI.SUM, root=0)

# Only print the result in process 0
if rank == 0:
    for i in range(K):
        print("moment %02d; val, approx: %d %0.6f" % (i + 1, round(integral_sum[i]), integral_sum[i]))


Overwriting 2020_spring_sol09_pr02.py


Results: 

```
moment 01; val, approx: 1 1.000002
moment 02; val, approx: 2 2.000000
moment 03; val, approx: 6 6.000000
moment 04; val, approx: 24 24.000000
moment 05; val, approx: 120 120.000000
moment 06; val, approx: 720 720.000000
moment 07; val, approx: 5040 5040.000000
moment 08; val, approx: 40320 40320.000000
moment 09; val, approx: 362880 362880.000000
moment 10; val, approx: 3628800 3628800.000000
moment 11; val, approx: 39916800 39916800.000000
moment 12; val, approx: 479001600 479001600.000000
moment 13; val, approx: 6227020800 6227020799.999996
moment 14; val, approx: 87178291200 87178291200.000168
moment 15; val, approx: 1307674368000 1307674368000.008301```


**Bonus Question: 10p** 

By definition $\Gamma(k) = \int_{0}^\infty x^{k - 1}e^{-x}dx$, hence $I_k = \Gamma(k + 1) = k!$. Moreover one can show it as follows. 


By using the partial integration, for every $k \geq 1$ it yields
$$\int_{0}^\infty x^{k}e^{-x}dx = \left[x^k e^{-k}\right]_0^{\infty} + k \int_{0}^\infty x^{k - 1}e^{-x}dx.$$


Note that $\left[x^k e^{-k}\right]_0^{\infty} = 0$, so the relation between the two consecutive moments is given by $I_{k} = k I_{k - 1}$. From $I_0 =\int_0^\infty e^{-x} = 1$ (moreover $I_0 = 1$ as the integral of a probability density function), finally we obtain $I_k = k!$







